# SQL 2 - multiple tables

In [1]:
import mysql.connector
import pandas as pd

In [2]:
import mysql.connector
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()  # loads variables from .env file
mysql_password = os.getenv("MYSQL_PASSWORD")

connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password=mysql_password,
)
cursor = connection.cursor()

In [3]:
cursor.execute("CREATE DATABASE IF NOT EXISTS AccountsMk2;")
cursor.execute("USE AccountsMk2;")

In [4]:
cursor.execute("DROP TABLE IF EXISTS Products, Customers, Orders;")
cursor.execute("CREATE TABLE Products (id INT AUTO_INCREMENT PRIMARY KEY, item VARCHAR(20), price INT, hub VARCHAR(20), binLocation VARCHAR(20), stockQty INT);")
cursor.execute("CREATE TABLE Customers (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(20), location VARCHAR(20));")
cursor.execute("CREATE TABLE Orders (id INT AUTO_INCREMENT PRIMARY KEY, customerId INT, itemId INT, qty INT, FOREIGN KEY (customerId) REFERENCES Customers(id), FOREIGN KEY (itemId) REFERENCES Products(id));")

In [5]:
cursor.execute('INSERT INTO Products (item, price, hub, binLocation, stockQty) VALUES '
               '("Teapot", 10, "Hull", "H34", 25),'
               '("Bicycle", 450, "Sheffield", "L36", 3),'
               '("TV", 500, "Sheffield", "A46", 2),'
               '("Sofa", 1050, "Sheffield", NULL, 1),'
               '("Kettle", 25, "Hull", "K102", 23),'
               '("Chair", 55, "Hull", "C3", 12),'
               '("Toaster", 32, "Leeds", "T104", 13),'
               '("Dustpan", 4, "Hull", NULL, 30);')
cursor.execute('INSERT INTO Customers (name, location) VALUES '
               '("Pablo", "London"),'
               '("Sophie", "Liphook"),'
               '("Sabina", "London");')
cursor.execute('INSERT INTO Orders (customerId, itemId, qty) VALUES '
               '(1, 1, 1),'
               '(1, 2, 1),'
               '(3, 6, 4),'
               '(3, 4, 1),'
               '(2, 1, 1);')
connection.commit()

In [6]:
# Join Orders with Customers and Products
cursor.execute("""
               SELECT c.name AS Customer, p.item AS Item, o.qty AS Quantity
               FROM Orders o
                        JOIN Customers c ON o.customerId = c.id
                        JOIN Products p ON o.itemId = p.id
               ORDER BY c.name, p.item;
               """)

# Fetch results
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

# Convert to DataFrame for nice display
df = pd.DataFrame(rows, columns=columns)
df

,Customer,Item,Quantity
0,Pablo,Bicycle,1
1,Pablo,Teapot,1
2,Sabina,Chair,4
3,Sabina,Sofa,1
4,Sophie,Teapot,1


In [7]:
cursor.execute("SELECT * FROM Products;")

# Fetch results
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

# Convert to DataFrame for nice display
df = pd.DataFrame(rows, columns=columns)
df

,id,item,price,hub,binLocation,stockQty
0,1,Teapot,10,Hull,H34,25
1,2,Bicycle,450,Sheffield,L36,3
2,3,TV,500,Sheffield,A46,2
3,4,Sofa,1050,Sheffield,None,1
4,5,Kettle,25,Hull,K102,23
5,6,Chair,55,Hull,C3,12
6,7,Toaster,32,Leeds,T104,13
7,8,Dustpan,4,Hull,None,30


## Conditional operators

### Equality operators

```=``` means equal to

```<>``` means NOT equal to

```>``` means greater than

```<``` means less than

```>=``` means greater than or equal to

```<=``` means less than or equal to

In [8]:
cursor.execute('''SELECT * FROM Products WHERE Price >= 50;''')
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,id,item,price,hub,binLocation,stockQty
0,2,Bicycle,450,Sheffield,L36,3
1,3,TV,500,Sheffield,A46,2
2,4,Sofa,1050,Sheffield,None,1
3,6,Chair,55,Hull,C3,12


### Other WHERE clause operators

```AND``` means when all conditions are met

```OR``` means when either condition is met

```BETWEEN``` combines multiple equality operators, e.g. ```>=``` and ```<=```

```IN``` enables simplification of multiple WHERE clauses, e.g. ```WHERE item IN ('Teapot', 'Chair', 'Toaster')```

```IS``` is an equality operator for expressions involving null

```IS NOT``` is an equality operator for expressions involving null

```LIKE``` enables imprecise matching using strings and wildcards, e.g. ```LIKE 'H%'``` for records starting with ```H```

```NOT LIKE``` enables imprecise inverse matching using strings and wildcards, e.g. ```NOT LIKE 'H%'``` for records NOT starting with ```H```

In [9]:
cursor.execute('''SELECT *
                  FROM Products
                  WHERE hub IN ('Hull', 'Leeds');
               ''')
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,id,item,price,hub,binLocation,stockQty
0,1,Teapot,10,Hull,H34,25
1,5,Kettle,25,Hull,K102,23
2,6,Chair,55,Hull,C3,12
3,7,Toaster,32,Leeds,T104,13
4,8,Dustpan,4,Hull,None,30


In [10]:
cursor.execute("""
               SELECT item, price, hub
               FROM Products
               WHERE binLocation IS NULL;
               """)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,item,price,hub
0,Sofa,1050,Sheffield
1,Dustpan,4,Hull


In [11]:
cursor.execute("""
               SELECT item, price, hub
               FROM Products
               WHERE binLocation IS NOT NULL;
               """)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,item,price,hub
0,Teapot,10,Hull
1,Bicycle,450,Sheffield
2,TV,500,Sheffield
3,Kettle,25,Hull
4,Chair,55,Hull
5,Toaster,32,Leeds


#### Examples of LIKE

In [20]:
starts_with_T_query = """
               SELECT item, price, hub
               FROM Products
               WHERE item LIKE 'T%';
               """
ends_with_e_query = """
               SELECT item, price, hub
               FROM Products
               WHERE item LIKE '%e';
               """
substring_le_query = """
               SELECT item, price, hub
               FROM Products
               WHERE item LIKE '%le%';
               """ # case-insensitive and anywhere in the entry

In [15]:
cursor.execute(starts_with_T_query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,item,price,hub
0,Teapot,10,Hull
1,TV,500,Sheffield
2,Toaster,32,Leeds


In [17]:
cursor.execute(ends_with_e_query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,item,price,hub
0,Bicycle,450,Sheffield
1,Kettle,25,Hull


In [21]:
cursor.execute(substring_le_query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,item,price,hub
0,Bicycle,450,Sheffield
1,Kettle,25,Hull


In [28]:
second_char_is_o_query = """
               SELECT item, price, hub
               FROM Products
               WHERE item LIKE '_o%';
               """
six_chars_and_second_is_e_query = """
               SELECT item, price, hub
               FROM Products
               WHERE item LIKE '_e____';
               """
contains_t_followed_by_3_chars_query = """
               SELECT item, price, hub
               FROM Products
               WHERE item LIKE '%T___';
               """

In [23]:
cursor.execute(second_char_is_o_query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,item,price,hub
0,Sofa,1050,Sheffield
1,Toaster,32,Leeds


In [25]:
cursor.execute(six_chars_and_second_is_e_query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,item,price,hub
0,Teapot,10,Hull
1,Kettle,25,Hull


In [29]:
cursor.execute(contains_t_followed_by_3_chars_query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,item,price,hub
0,Kettle,25,Hull
1,Dustpan,4,Hull
